In [48]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=128)
test_dataloader = DataLoader(test_data, batch_size=100)


Files already downloaded and verified
Files already downloaded and verified


In [49]:
vgg_configs = {
'A': [64,'P',128,'P',256,256,'P',512,512,'P',512,512,'P'],
'B': [64,64,'P',128,128,'P',256,256,'P',512,512,'P',512,512,'P'],
'C': [64,64,'P',128,128,'P',256,256,'conv1','P',512,512,'conv1','P',512,512,'conv1','P'],
'D': [64,64,'P',128,128,'P',256,256,256,'P',512,512,512,'P',512,512,512,'P'],
'E': [64,64,'P',128,128,'P',256,256,256,256,'P',512,512,512,512,'P',512,512,512,512,'P'],
'F': [64,64,'P',128,128,'P',256,256,'P',512,512,512,512,512,'P'] #to test variations 
}

class VGG(nn.Module):
    def __init__(self,config):
        super(VGG, self).__init__()
        self.config = config
        self.convs = self.stacklayers()
        self.fullyconnected = nn.Sequential(nn.Linear(512,256),nn.ReLU(),nn.Dropout(),nn.Linear(256,256),nn.ReLU(),nn.Dropout(),nn.Linear(256,10))

    def forward(self, x):
        x = self.convs(x)
        x = nn.Flatten()(x)
        logits = self.fullyconnected(x)
        return logits

    def stacklayers(self):
        res = []
        prev = 3
        for layer in vgg_configs[self.config]:
            if layer == 'P':
              res.append(nn.MaxPool2d(2,2))
            elif layer == 'conv1':
              res += [nn.Conv2d(prev,prev,1),nn.BatchNorm2d(prev),nn.ReLU()]
            else:
              res += [nn.Conv2d(prev,layer,3,padding='same'),nn.BatchNorm2d(layer),nn.ReLU()]
              prev = layer
        return nn.Sequential(*res)


In [50]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [51]:
learning_rate = 0.01
loss_fn = nn.CrossEntropyLoss()
model = VGG('E')
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,momentum=0.9, weight_decay=5*0.0001)
model = model.to(device)

epochs = 50
for t in range(epochs):
    if t%10 == 0 and t != 0:
      learning_rate = max(learning_rate/3.3,0.0001)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,momentum=0.9, weight_decay=5*0.0001)
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    
print("Done!")

Epoch 1
-------------------------------
loss: 2.375035  [    0/50000]
loss: 1.702620  [12800/50000]
loss: 1.587500  [25600/50000]
loss: 1.271972  [38400/50000]
Test Error: 
 Accuracy: 56.4%, Avg loss: 1.249890 

Epoch 2
-------------------------------
loss: 1.090797  [    0/50000]
loss: 1.108820  [12800/50000]
loss: 0.967657  [25600/50000]
loss: 0.869436  [38400/50000]
Test Error: 
 Accuracy: 68.5%, Avg loss: 0.965175 

Epoch 3
-------------------------------
loss: 0.838702  [    0/50000]
loss: 0.810136  [12800/50000]
loss: 0.855137  [25600/50000]
loss: 0.696152  [38400/50000]
Test Error: 
 Accuracy: 72.4%, Avg loss: 0.865409 

Epoch 4
-------------------------------
loss: 0.647445  [    0/50000]
loss: 0.728626  [12800/50000]
loss: 0.717536  [25600/50000]
loss: 0.595837  [38400/50000]
Test Error: 
 Accuracy: 74.9%, Avg loss: 0.791614 

Epoch 5
-------------------------------
loss: 0.468625  [    0/50000]
loss: 0.607054  [12800/50000]
loss: 0.550023  [25600/50000]
loss: 0.490074  [38400